<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [24]:
import pandas as pd
import jaydebeapi
import json
import cx_Oracle

In [2]:
path = "access to the database.txt"
with open(path) as f:
    access = json.load(f)
    
driver = 'ojdbc14.jar'
path_base = access['path_base']
password = access['password']
login = access['login']
port = access['port']
sid = access['sid']

conn = jaydebeapi.connect(
    'oracle.jdbc.driver.OracleDriver',
    f'jdbc:oracle:thin:{login}/{password}@{path_base}:{port}/{sid}',
    [login, password],
    driver)

curs = conn.cursor()

In [ ]:
def addTable():
    curs.execute(
        '''insert into uszn.temp$_vozv 
           (F, I, O, SUMA, PR)
            values(1, 'привет', '3', '4', '5')''')
    
def delTable():
    curs.execute('''delete from uszn.temp$_vozv''')    

In [ ]:
def ShowCountTable():
    curs.execute('''select count(*) from uszn.temp$_vozv''')
    print(curs.fetchall())
    
def ShowTable():
    curs.execute('''select * from uszn.tsrv_user_groups''')
    for row in curs.fetchall():
        print(row)
    


In [ ]:
ShowTable()

In [ ]:
delTable()

In [ ]:
curs.execute('''
    select *
        from uszn.temp$_vozv
    ''')
for row in curs.fetchall():
    print(row)

In [ ]:
curs.execute('''select count(*) from uszn.temp$_vozv''')
print(curs.fetchall())

In [29]:
def zags():
    curs.execute('''
    select 
       REPLACE(uszn.pkGen.BLOBToCLOB(msg.message_data, 'UTF8'),'<?xml version="1.0" encoding="UTF-8" ?>')
    from uszn.all_smev3_inc_messages msg
where
  message_kind_id=1 and
  data_kind_region_id=0 and
  data_kind_id in (
    /* старые форматы */ 16, 17, 18, 19, 20,
    /* новые форматы */ 28, 29, 30, 31, 32, 33) and
  Trunc(date_created, 'dd') between ADD_MONTHS(TRUNC(sysdate,'mm'),-1) and sysdate
order by date_created, id
    ''')
    with open('zagz.xml', 'w') as f:
        f.write('<?xml version="1.0" encoding="UTF-8" ?>')
        f.write('<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/">')
        for row in curs.fetchall():
            f.write('\n')
#             f.write(row[0] )
            print(str(row[0][0]))
            n = input()
            if n == '1':
                break
        f.write('\n')
        f.write('</SOAP-ENV:Envelope>')

In [30]:
zags()

TypeError: 'oracle.sql.CLOB' object is not subscriptable

In [40]:
def alarm_ep():
    curs.execute(
    '''select count(*) from
     (select id
       from uszn.all_smev3_inc_messages
       where date_created>=To_Date('01.01.2021') and proc_status_id in (3))'''
    )
    cnt = int(curs.fetchall()[0][0])
    # print(cnt, type(cnt))
    if cnt > 0:
        curs.execute(
        '''
        declare
          iiIDs        uszn.pkGen.TIntegers;
          i$ Pls_Integer;
        begin
        select id bulk collect into iiIDs
          from uszn.all_smev3_inc_messages
          where date_created>=To_Date('01.01.2021') and proc_status_id in (3);
          for i in 1..iiIDs.count loop
            begin
              i$ := uszn.pkSMEV3.ProcessInMessage(iiIDs(i), 1, 1,'Принудительная обработка входящего сообщения СМЭВ-3');
            commit;
            exception
              when Others then
                rollback;
                raise;
            end;
          end loop;
        end;
        '''
        )

In [41]:
alarm_ep()